In [0]:
# Importations
import sys      
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup, Tag, NavigableString, Comment
import pickle
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from nltk.corpus import stopwords
from google.colab import drive
import tensorflow as tf
import pandas as pd

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [0]:
tf.enable_eager_execution()

In [0]:
#drive.mount('/content/gdrive')
#!ls '/content/gdrive/My Drive/Text_mining_projet'
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'
downloaded = drive.CreateFile({'id': '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'})
downloaded.GetContentFile('OpenSubtitles.de-fr.fr')

In [0]:
text = open("OpenSubtitles.de-fr.fr", "r").read() #.decode(encoding='utf-8')

In [0]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 558646528 characters


In [0]:
ncarac = int(len(text)/8)#int(len(text)/3)
ncarac

186215509

In [0]:
text = text[:ncarac]

In [0]:
#type(corpus_conv)

In [0]:
print(text[:250])

Bonjour. Je suis rédacteur au journal L'Illustration.
Je cherche M. Odilon Hubert, député d'Alger.
Il est là-bas.
Merci.
Monsieur Hubert ?
Camille Bruyère.
Vous êtes le 1er représentant des Français d'Algérie au Parlement.
Mon journal voudrait vous c


In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

288 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
with open("idx2carCOURT", "wb") as fin :
  pickle.dump(idx2char, fin)

In [0]:
with open("car2idxCOURT", "wb") as fin :
  pickle.dump(char2idx, fin)

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Bonjour. Je s' ---- characters mapped to int ---- > [32 77 76 72 77 83 80 14  1 40 67  1 81]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

B
o
n
j
o


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"
" est là-bas.\nMerci.\nMonsieur Hubert ?\nCamille Bruyère.\nVous êtes le 1er représentant des Français d'A"
"lgérie au Parlement.\nMon journal voudrait vous consacrer un article.\nSur l'Algérie ?\nNon. Sur vous.\nS"
"ur moi ?\nC'est une histoire de violence et de sang, à la Shakespeare.\nLes lecteurs adorent ça, monsie"
"ur.\nVous croyez ?\nIl s'agit d'un autre pays, jeune homme, d'un autre monde, d'une autre terre où tout"


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "Bonjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nI"
Target data: "onjour. Je suis rédacteur au journal L'Illustration.\nJe cherche M. Odilon Hubert, député d'Alger.\nIl"


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 32 ('B')
  expected output: 77 ('o')
Step    1
  input: 77 ('o')
  expected output: 76 ('n')
Step    2
  input: 76 ('n')
  expected output: 72 ('j')
Step    3
  input: 72 ('j')
  expected output: 77 ('o')
Step    4
  input: 77 ('o')
  expected output: 83 ('u')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 288) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           73728     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 288)           295200    
Total params: 4,304,160
Trainable params: 4,304,160
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([188, 273,  97, 218, 117, 186, 131,  57,  28, 241, 109,  40, 253,
        37,  19,  47, 189,  92, 117, 250, 147, 107, 235, 155, 273, 242,
       215,  15,  36, 260,  81, 149, 232,  31, 209,  80,  29, 121,  77,
       275, 112, 249, 225,  80,  12, 152,  39, 167,  57, 226,  87, 170,
       283, 118, 136,  44, 131, 252, 286, 110,  36,  98, 260, 135,  13,
       209,  11, 217, 262, 178, 220, 257,   2, 281, 251,  91,  51, 216,
        75, 225,  56, 152, 176, 137,  84, 202, 249, 226,  51, 102, 140,
       110, 263, 232, 205, 145, 229, 168,  83,  57])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'uand nous marions-nous?\nBientôt.\nAussi tôt que tu voudras.\nDepuis trois générations, cette couronne '

Next Char Predictions: 
 'êⰰ\x86Œ\x9dè\xad[=Ρ\x94JξG3Që\x80\x9dι¿\x91ΕÇⰰΤŁ/F\u200esÁΑAÿr?¡oⴭ\x97θůr,ÄIÓ[űy×ﬀ\x9e³N\xadν\ufeff\x95F\x88\u200e²-ÿ+ń—àŔϋ!㤹κ~UłmůZÄÞ´vøθűU\x8c·\x95‚Αû½žÔu['


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 288)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.663828


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=1#5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
   9523/Unknown - 1799s 189ms/step - loss: 1.2771

In [0]:
#model.save('model_caract.h5')

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Bonjour"))

In [0]:
model.save('model_caractCOURT.h5')